<a href="https://colab.research.google.com/github/Yoonsun222/CapstoneDesign/blob/main/%EC%BA%A1%EC%8A%A4%ED%86%A4%EB%94%94%EC%9E%90%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [40]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df1  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.01.31-02.06.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df2  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.07-02.13.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df3  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.14-02.20.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df4 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.21-02.27.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df5 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.28-03.06.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
weather_df = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/기상청데이터.csv", encoding = 'euc-kr', parse_dates=['등록일자'])

df = pd.concat([df1,df2,df3,df4,df5],axis=0, join='inner')



In [43]:
df['등록일자'] = df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

df

,시리얼,관측시각,온도(℃),습도(%),미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),등록일자
0,V10G1802191,2.021050e+11,20,49,5,4,99,405,2022-01-31 00:00
1,V10G1802099,2.021050e+11,21,46,4,3,106,447,2022-01-31 00:00
2,V10G1802225,2.021050e+11,21,49,4,3,27,461,2022-01-31 00:00
3,V10G1802415,2.021050e+11,17,49,4,3,581,383,2022-01-31 00:00
4,V10G1802677,2.021050e+11,21,46,5,4,7,570,2022-01-31 00:00
...,...,...,...,...,...,...,...,...,...
642691,V10G1803626,2.021050e+11,22,49,3,2,143,703,2022-03-06 23:00
642692,V10G1802586,2.021050e+11,21,44,3,3,10,372,2022-03-06 23:00
642693,V10G1802051,2.021050e+11,21,49,4,3,25,69,2022-03-06 23:00
642694,V10G1802859,2.021050e+11,21,61,3,3,4,383,2022-03-06 23:00


In [41]:
weather_df['등록일자'] = weather_df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

weather_df

,등록일자,기온(°C),풍속(m/s),습도(%),일조(hr),전운량(10분위)
0,2022-01-31 01:00,-4.6,2.4,51,0.0,0
1,2022-01-31 02:00,-5.0,2.3,52,0.0,0
2,2022-01-31 03:00,-5.3,1.6,55,0.0,0
3,2022-01-31 04:00,-5.8,1.0,55,0.0,0
4,2022-01-31 05:00,-6.1,1.9,58,0.0,0
...,...,...,...,...,...,...
811,2022-03-05 20:00,1.5,3.9,37,0.0,0
812,2022-03-05 21:00,0.7,3.3,40,0.0,0
813,2022-03-05 22:00,0.0,2.9,40,0.0,0
814,2022-03-05 23:00,-0.3,3.0,41,0.0,0


In [67]:
data = pd.merge(left = df , right = weather_df, how = "inner", on = "등록일자")
data.head(100)


#temp = df[df['시리얼']== 'V01T1622211']

total_data = data[data.columns[2:]]
total_data = total_data.replace(-9999,np.nan)
total_data = total_data.fillna(method='bfill')
total_data.head(4)


In [ ]:
X =  total_data[total_data.columns[[0,1,4,5,7,8,9,10,11]]]
Y = total_data[total_data.columns[[2,3]]]


In [68]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, random_state=22)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)

X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)


X_train_scaled

,온도(℃),습도(%)_x,휘발성유기화합물(ppb),이산화탄소(ppm),기온(°C),풍속(m/s),습도(%)_y,일조(hr),전운량(10분위)
3469290,0.960938,0.472527,0.013060,0.1780,0.972,0.461538,0.380282,0.8,0.0
3008255,0.953125,0.538462,0.253148,0.3428,0.576,0.205128,0.873239,0.0,0.8
3438297,0.976562,0.450549,0.001329,0.1520,0.468,0.282051,0.549296,0.0,0.0
1293807,0.953125,0.516484,0.000391,0.2296,0.520,0.294872,0.760563,0.0,0.9
2684539,0.945312,0.494505,0.001173,0.1136,0.316,0.000000,0.915493,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1212404,0.953125,0.472527,0.000547,0.1564,0.364,0.217949,0.774648,1.0,0.0
292105,0.960938,0.527473,0.000938,0.2128,0.308,0.282051,0.338028,0.0,0.0
1696495,0.953125,0.527473,0.001408,0.1704,0.148,0.384615,0.338028,0.0,0.0
961901,0.945312,0.560440,0.001095,0.1080,0.352,0.179487,0.661972,0.0,0.0
